In [ ]:
!pip install pyupbit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112 kB 11.3 MB/s 


In [ ]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyupbit
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, accuracy_score, recall_score, precision_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import lightgbm
import pickle

In [ ]:
df = pyupbit.get_ohlcv("KRW-BTC", interval = 'minute1', count = 20000)
df

,open,high,low,close,volume,value
2022-08-15 01:37:00,32627000.0,32668000.0,32624000.0,32626000.0,6.614164,2.159122e+08
2022-08-15 01:38:00,32625000.0,32664000.0,32613000.0,32619000.0,10.716572,3.496712e+08
2022-08-15 01:39:00,32611000.0,32661000.0,32556000.0,32564000.0,12.550235,4.090182e+08
2022-08-15 01:40:00,32564000.0,32580000.0,32551000.0,32580000.0,6.426242,2.092741e+08
2022-08-15 01:41:00,32579000.0,32582000.0,32572000.0,32582000.0,1.976333,6.438668e+07
...,...,...,...,...,...,...
2022-08-28 22:58:00,27561000.0,27566000.0,27561000.0,27566000.0,0.550219,1.516614e+07
2022-08-28 22:59:00,27566000.0,27567000.0,27561000.0,27566000.0,0.821752,2.265071e+07
2022-08-28 23:00:00,27567000.0,27567000.0,27562000.0,27567000.0,0.434002,1.196358e+07
2022-08-28 23:01:00,27567000.0,27567000.0,27562000.0,27567000.0,0.269857,7.438965e+06


In [ ]:
ml = df.reset_index()
ml.rename(columns={'index':'time'},inplace = True)
ml

,time,open,high,low,close,volume,value
0,2022-08-15 01:37:00,32627000.0,32668000.0,32624000.0,32626000.0,6.614164,2.159122e+08
1,2022-08-15 01:38:00,32625000.0,32664000.0,32613000.0,32619000.0,10.716572,3.496712e+08
2,2022-08-15 01:39:00,32611000.0,32661000.0,32556000.0,32564000.0,12.550235,4.090182e+08
3,2022-08-15 01:40:00,32564000.0,32580000.0,32551000.0,32580000.0,6.426242,2.092741e+08
4,2022-08-15 01:41:00,32579000.0,32582000.0,32572000.0,32582000.0,1.976333,6.438668e+07
...,...,...,...,...,...,...,...
19995,2022-08-28 22:58:00,27561000.0,27566000.0,27561000.0,27566000.0,0.550219,1.516614e+07
19996,2022-08-28 22:59:00,27566000.0,27567000.0,27561000.0,27566000.0,0.821752,2.265071e+07
19997,2022-08-28 23:00:00,27567000.0,27567000.0,27562000.0,27567000.0,0.434002,1.196358e+07
19998,2022-08-28 23:01:00,27567000.0,27567000.0,27562000.0,27567000.0,0.269857,7.438965e+06


In [ ]:
ml['gap'] = ml['close'] - ml['close'].shift(1)
ml['upgap'] = np.where(ml['gap']>=0, ml['gap'], 0)
ml['downgap'] = np.where(ml['gap'] <0, ml['gap'].abs(), 0)




ml['AU'] = ml['upgap'].ewm(alpha=1/10, min_periods=10).mean()
ml['AD'] = ml['downgap'].ewm(alpha=1/10, min_periods=10).mean()

ml['RSI'] = ml['AU'] / (ml['AU'] + ml['AD']) * 100



ml['ma10'] = ml['close'].rolling(window=10).mean() # 10분 이평
ml['stddev'] = ml['close'].rolling(window=10).std() # 10분 이평편차
ml['upper'] = ml['ma10'] + 2*ml['stddev'] # 상단밴드
ml['lower'] = ml['ma10'] - 2*ml['stddev'] # 하단밴드
ml['bandwidth'] = ml['upper'] - ml['lower']


ml['disparity'] = 100*(ml['close']/ml['ma10'])

ml['up_down'] = (ml['close'] - ml['close'].shift(1)) > 0

ml['buy_sell'] = (ml['gap'].rolling(window=5).sum()) > 0

ml.drop(index=[0, 1, 2, 3, 4, 5, 6, 7, 8], axis=0, inplace = True)
ml.reset_index(inplace = True, drop = True)

In [ ]:
db = ml.copy()
ml.drop(['time','AU','AD','upgap','downgap','upper','lower','gap','ma10','stddev','up_down'], axis = 1, inplace = True)

In [ ]:
ml.head(50)

,open,high,low,close,volume,value,RSI,bandwidth,disparity,buy_sell
0,32558000.0,32610000.0,32553000.0,32594000.0,3.304722,1.076663e+08,49.211872,100150.997110,100.033760,True
1,32603000.0,32603000.0,32546000.0,32575000.0,3.690925,1.201944e+08,41.863201,79965.548137,99.991098,False
2,32552000.0,32575000.0,32519000.0,32554000.0,7.012522,2.282000e+08,35.375829,60463.395722,99.946579,False
3,32540000.0,32553000.0,32500000.0,32503000.0,14.585440,4.741066e+08,24.944841,105898.064194,99.808692,False
4,32526000.0,32544000.0,32504000.0,32537000.0,6.974704,2.267577e+08,38.399414,109203.174557,99.926292,False
5,32537000.0,32539000.0,32509000.0,32536000.0,2.656829,8.639238e+07,38.175770,108978.285299,99.937340,False
6,32536000.0,32574000.0,32516000.0,32567000.0,2.754783,8.965975e+07,48.505940,104715.487552,100.036861,False
7,32572000.0,32575000.0,32521000.0,32531000.0,2.101618,6.841576e+07,39.902907,104565.981296,99.939786,False
8,32531000.0,32573000.0,32527000.0,32562000.0,4.260759,1.386670e+08,48.621644,104701.904896,100.028876,True
9,32540000.0,32567000.0,32511000.0,32545000.0,5.945447,1.933997e+08,44.672649,103007.227362,99.983410,True


In [ ]:
db.head()

,time,open,high,low,close,volume,value,gap,upgap,downgap,...,AD,RSI,ma10,stddev,upper,lower,bandwidth,disparity,up_down,buy_sell
0,2022-08-15 01:46:00,32558000.0,32610000.0,32553000.0,32594000.0,3.304722,1.076663e+08,27000.0,27000.0,0.0,...,9377.810245,49.211872,32583000.0,25037.749278,3.263308e+07,3.253292e+07,100150.997110,100.033760,True,True
1,2022-08-15 01:47:00,32603000.0,32603000.0,32546000.0,32575000.0,3.690925,1.201944e+08,-19000.0,0.0,19000.0,...,10780.074649,41.863201,32577900.0,19991.387034,3.261788e+07,3.253792e+07,79965.548137,99.991098,False,False
2,2022-08-15 01:48:00,32552000.0,32575000.0,32519000.0,32554000.0,7.012522,2.282000e+08,-21000.0,0.0,21000.0,...,12204.314618,35.375829,32571400.0,15115.848930,3.260163e+07,3.254117e+07,60463.395722,99.946579,False,False
3,2022-08-15 01:49:00,32540000.0,32553000.0,32500000.0,32503000.0,14.585440,4.741066e+08,-51000.0,0.0,51000.0,...,17406.109664,24.944841,32565300.0,26474.516048,3.261825e+07,3.251235e+07,105898.064194,99.808692,False,False
4,2022-08-15 01:50:00,32526000.0,32544000.0,32504000.0,32537000.0,6.974704,2.267577e+08,34000.0,34000.0,0.0,...,15149.187238,38.399414,32561000.0,27300.793639,3.261560e+07,3.250640e+07,109203.174557,99.926292,True,False


In [ ]:
target = 'buy_sell'

train = ml.iloc[0:11940,:]
val = ml.iloc[11941:15921,:]
test = ml.iloc[15922:,:]

X_train = train.iloc[:,:-1]
X_val = val.iloc[:,:-1]
X_test = test.iloc[:,:-1]

y_train = train[target]
y_val = val[target]
y_test = test[target]

In [ ]:
lgb_clf = lightgbm.LGBMClassifier(objective='binary', random_state = 11)

tscv = TimeSeriesSplit(n_splits = 5)

param = {
    'n_estimators':[180,200],
    'max_depth':[6,9,25],
    'num_leaves':[5,10,15],
    'min_child_samples':[320,380,400]   
}

grid_lgb = GridSearchCV(lgb_clf,                        # 모델 객체
                  param_grid = param,     # 하이퍼파라미터 목록
                  scoring = 'accuracy',
                  cv = tscv,
                  n_jobs = -1
                                   
                  )


grid_lgb.fit(X_train, y_train)

bestgrid = grid_lgb.best_estimator_
bestgridpred = bestgrid.predict(X_val)

print("최적 파라미터 : ", grid_lgb.best_params_, '\n\n',
      'best score :', grid_lgb.best_score_, '\n\n',
      'best f1 :', f1_score(y_val, bestgridpred)
      )

최적 파라미터 :  {'max_depth': 6, 'min_child_samples': 380, 'n_estimators': 180, 'num_leaves': 5} 

 best score : 0.8548743718592966 

 best f1 : 0.8529870129870128


In [ ]:
bestgrid.fit(X_test, y_test)
test_pred = bestgrid.predict(X_test)

print('test accuracy score :', accuracy_score(y_test, test_pred), '\n\n',
      '     test f1 score :', f1_score(y_test, test_pred)
      )

test accuracy score : 0.8758908822806586 

      test f1 score : 0.866296002118083


In [ ]:
with open('model.pkl','wb') as pickle_file:
    pickle.dump(bestgrid, pickle_file)

In [ ]:
import pickle

with open('model.pkl','wb') as pickle_file:
    pickle.dump(model, pickle_file)